# Custom validators

In [ ]:
!lamin init --storage test-validator --schema bionty

In [ ]:
import lamindb as ln
import lnschema_bionty as lb
from lamin_utils import logger

lb.settings.organism = "human"

Let's use an AnnData as the dataset to validate:

In [ ]:
adata = ln.dev.datasets.anndata_human_immune_cells(populate_registries=True)
adata

Define validation criteria for an `AnnData`:

In [ ]:
validators = {
    "var": {"index": lb.Gene.ensembl_gene_id},
    "obs": {
        "donor": ln.ULabel.name,
        "tissue": lb.Tissue.name,
        "cell_type": lb.CellType.name,
        "assay": lb.ExperimentalFactor.name,
    },
}

Run bulk validation:

In [ ]:
features = ln.Feature.lookup()

for slot, slot_validators in validators.items():
    for name, validator in slot_validators.items():
        # registry model
        model = validator.field.model

        # validate index
        if name == "index":
            logger.print(f"validating {slot}.{name}:")
            index = adata.__getattribute__(slot).index
            validated = model.validate(index, validator)
            if validated.sum() == len(index):
                logger.print("🎉 PASS")

        # validate columns
        else:
            logger.print(f"\nvalidating {slot}.{name}:")
            # check if the column name exist
            if name not in adata.__getattribute__(slot).columns:
                logger.warning(f"{slot}.{name} field is missing")
            else:
                # check if a feature is registered for the column
                if not hasattr(features, name):
                    logger.warning(f"feature '{name}' is not registered")
                # validate categorical labels in a column
                else:
                    labels = adata.__getattribute__(slot)[name]
                    validated = model.validate(labels, validator)
                    if validated.sum() == len(labels):
                        logger.print("🎉 PASS")

In [ ]:
!lamin delete --force test-validator